# Introduction to classification in python  

Developped for Remote sensing image processing course  
Department of Applied Geoinformatics and Carthography, Faculty of Science, Charles University  
Author: Adam Kulich, 2025  

library requirements: gdal, numpy, matplotlib, scikit-learn

In [2]:
import numpy as np
from osgeo import gdal, ogr
import matplotlib.pyplot as plt
import sklearn

#### Opening and visualization of the image  

For any kind of image analysis, it needs to be converted into a multidimensional matrix containing the DN values of the individual pixels in each band. For this step, we use the gdal library.


In [ ]:
#open image using gdal
image_path = "INSERT_IMAGE_PATH.tif"
image = gdal.Open(image_path)

#you can check image properties
n_col = image.RasterXSize
n_row = image.RasterYSize
n_bands = image.RasterCount
print(f"Width: {n_col}, Height: {n_row}, Bands: {n_bands}")

#or georeferencing information
projection = image.GetProjection() 
geotransform = image.GetGeoTransform()
print(f"Projection: {projection}") 
print(f"GeoTransform: {geotransform}")

#see more about GeoTransform attribute at https://gdal.org/en/stable/tutorials/geotransforms_tut.html

Now when we opened the image and opened its properties, we can rewrite it into an array of 8-bit integers.

In [ ]:
#read the bands as arrays
red = image.GetRasterBand(1).ReadAsArray().astype(np.uint8) 
green = image.GetRasterBand(2).ReadAsArray().astype(np.uint8) 
blue = image.GetRasterBand(3).ReadAsArray().astype(np.uint8)

#next it is needed to stack it in one array
rgb_array = np.dstack((red, green, blue))
#let's check the value at a specific place
print("Value of red band at 120th row and 120th column: {rgb_array[120,120,0]}")

Now we can display the image with matplotlib.

In [ ]:
plt.imshow(rgb_array)
plt.title("RGB Composite")
plt.axis("off")
plt.show()

#### Extracting data from the image for training, validation and testing  
To open vectors, we use org, which is part of gdal library.

In [ ]:
#opening the vector with ogr
vector_path = "INSERT_VECTOR_PATH"
vector = ogr.Open(vector_path)
#extracting the feature layer from the file
vector_layer = vector.GetLayer()

#creating an empty raster (of the same shape as the image)
out_raster_ds = gdal.GetDriverByName("MEM").Create("", n_col, n_row, 1, gdal.GDT_Byte)

#filling the empty raster with vector mask (0 or classvalue)
status = gdal.RasterizeLayer(out_raster_ds, [1], vector.GetLayerByIndex(0), None, None, [0], ['ALL_TOUCHED=TRUE', f'ATTRIBUTE={classvalue}'])

#read the raster mask as an array
rasterized_vector = out_raster_ds.GetRasterBand(1).ReadAsArray()

#extract input data from the image and mask
X = rgb_array[rasterized_vector > 0, :]
#extract target data from the rasterized vector
y = rasterized_vector[rasterized_vector > 0, :]

#### First look at scikit-learn library
Let's explore the possibilities in the scikit-learn library: https://scikit-learn.org/stable/  

First, we need to separate train and test data, so we can evaluate the model:

In [ ]:
train_data, test_data, train_target, test_target = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

Can we do the same if the ground truth dataset is defined by polygons, not points?  

Now let's try to create a random forest model and evaulate it using test data.

In [ ]:
#it is needed to import parts of the sklearn library
import sklearn.ensemble
import sklearn.preprocessing

#for most data it is worth it to scale it between 0 and 1
scaler =  sklearn.preprocessing.MinMaxScaler()

#fitting the scaler and transforming the train input
scaler.fit_transform(train_data)
#transforming the test input
scaler.transform(test_data)

#initialize the random forest classifier
model = sklearn.ensemble.RandomForestClassifier()

#train the classifier on the train data
model.fit(train_data,test_data)

#predict the test data
prediction = model.predict(test_data)

#compute evaluation metrics and print the results
accuracy = sklearn.metrics.accuracy_score(test_target, prediction)
matrix = sklearn.metrics.confusion_matrix(test_target, prediction)
report = sklearn.metrics.classification_report(test_target, prediction, output_dict=True)
print("Final accuracy is", accuracy, "and F-1 score", report['weighted avg']['f1-score'])
print("Confusion matrix:", matrix)
print("Full report:", report)

#### Optimizing hyperparameters  
Next we can fine ture the model and improve the result a bit.  
First, let's show how we can set a hyperparameter. Using the scikit-learn documentation (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier), we can look at parameters of the RandomForestClassifier method. Lets change some of them.

In [ ]:
#initialize the random forest classifier with ntree=85, mtry=3 and maximum depth=2
model2 = sklearn.ensemble.RandomForestClassifier(n_estimators=150,max_features=3,max_depth=2)
model2.fit(train_data,test_data)
prediction2 = model2.predict(test_data)

accuracy2 = sklearn.metrics.accuracy_score(test_target, prediction2)
report2 = sklearn.metrics.classification_report(test_target, prediction2, output_dict=True)
print("Final accuracy is", accuracy2, "and F-1 score", report2['weighted avg']['f1-score'])

The result is probably worse than before - how can we find a better solution?

One option is grid search using cross-validation. This way we can compare different setting without touching the test set. Comparing the results of multiple evaluations on the test set could lead to overfitting. Scikit-learn library offers a built-in function for grid search.

In [ ]:
import sklearn.model_selection

#let's first define the parameters we need to search
param_grid = {
    'n_estimators': [50, 100, 200, 400],
    'max_features': [2, 3],
    'max_depth': [3, 5, 7, None],
    'min_samples_split': [2, 3, 6],
    'min_samples_leaf': [1, 3, 5]
}

#initialize Grid search
grid_search = sklearn.model_selection.GridSearchCV(
    estimator=sklearn.ensemble.RandomForestClassifier(),
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    verbose=1,
    n_jobs=-1
)

#fit the model
grid_search.fit(train_data, train_target)

#best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

Beware that the gridsearch might take very long if the grid is large and multidimensional. The example you just ran has to train and evaluate 864 models. This could take very long time if the train dataset is larger and there are more features. As a reasonable alternative, you can use sklearn.model_selection.RandomizedSearchCV().  

#### Final image classification

Now we can train the final model with the best parameters and classify the whole image.

In [ ]:
#train and evaluate the final model
final_model = sklearn.ensemble.RandomForestClassifier(grid_search.best_params_)
final_model.fit(train_data,test_data)
final_prediction = final_model.predict(test_data)

final_accuracy = sklearn.metrics.accuracy_score(test_target, final_prediction)
final_matrix = sklearn.metrics.confusion_matrix(test_target, final_prediction)
final_report = sklearn.metrics.classification_report(test_target, final_prediction, output_dict=True)
print("Final accuracy is", final_accuracy, "and F-1 score", final_report['weighted avg']['f1-score'])
print("Confusion matrix:", final_matrix)
print("Full report:", final_report)

#reshape the image into (n_samples, n_features) so it can be used as an imput to the model
rgb_array = rgb_array.reshape(-1, n_bands)
scaler.transform(rgb_array)

#predict whole array
classification = model.predict(rgb_array)
classification = classification.reshape(n_row, n_col)

#vizualize the classification
from matplotlib.colors import ListedColormap

#set color map
colors = ["#00B050", "#92D050", "#00B0F0", "#FF0000"]
cmap = ListedColormap(colors)

#create figure and display
plt.imshow(classification, cmap=cmap)
#add legend
cbar = plt.colorbar(ticks=[1, 2, 3, 4])
cbar.ax.set_yticklabels(["Forest", "Open grassland and crops", "Water body", "Urban area"])

plt.title("Classified Image")
plt.axis("off")  # Hide axes
plt.show()

It is also very useful to save the classified image to a new .tif file. This can be done using gdal again, with the information about the input image.

In [ ]:
#set output path
output_path = "SET_OUTPUT_PATH"

#choose memory driver
memory_driver = gdal.GetDriverByName('GTiff')

#create an empty raster and set the reference
out_raster = memory_driver.Create(output_path, n_col, n_row, n_bands, gdal.gdal.GDT_Byte)
out_raster.SetGeoTransform(geotransform)
out_raster.SetProjection(projection)

#write the classification array into the raster
out_raster_ds.GetRasterBand(1).WriteArray(classification)

Assignment - Classification playground:
- Classify the image using at least two other algorithms.
- You can use scikit-learn library (go through the documentation to find other methods) or some other you choose (for example XGBoost classifier has its own library).
- Try to achieve the highest possible accuracy (using the same test set, generated by "train_test_split(X, y, test_size=0.2, random_state=42)").
- Do not use the test set to tune hyperparameters! Only to evaluate the best model.
- Save the classification, accuracy, confusion matrix and classification report.
- Send a report to adam.kulich@natur.cuni.cz until XX.XX.2025 including:
  - Best accuracy and confusion matrix.
  - Detailed procedure used to achieve the result (including how you choose the methods you tried and why).
  - All methods you tried and comparison of their best results.
  - Resulting map of the land cover you made.